# Best practices for beamformer source reconstruction


`
Author: Britta Westner
`

`
parts of this notebook are inspired by the Beamformer tutorial of MNE-Python: https://mne.tools/stable/auto_tutorials/inverse/50_beamformer_lcmv.html#sphx-glr-auto-tutorials-inverse-50-beamformer-lcmv-py
`

In this notebook, we will have a look at best practices and pitfalls in beamformer source reconstruction. We use the `sample` dataset. 

The best practices discussed here are relevant knowledge for any beamformer application. Some more specific best practices are discussed in the following notebooks:
- Beamforming with EEG data: `Source_reconstruction_with_EEG.ipynb`
- Working with Maxwell-filtered data (or otherwise severly rank-deficient data): `Working_with_SSSed_data.ipynb`
- Working with multiple channel types at the same time: `Combining_channel_types.ipynb`

In [1]:
%matplotlib inline
import mne

mne.set_log_level('warning')

# set paths:
sample_path = mne.datasets.sample.data_path()
data_path = sample_path / 'MEG' / 'sample'
subjects_dir = sample_path / 'subjects'

## Read and epoch data

First, we read the raw data from disk. We only keep the gradiometer MEG data, as we are not interested in the EEG data for this tutorial and want to keep things simple by only looking at one MEG channel type. We also keep the stim channels, because we still want to cut the data in to epochs later. 

In [ ]:
# read raw data
fname_raw = data_path / 'sample_audvis_raw.fif'
raw = mne.io.read_raw_fif(fname_raw)
raw.pick(picks=['mag', 'stim'])

Let's create epochs from the continuous data and compute the evoked fields. We use only the epochs that use visual stimulation. From the epochs, we create the evoked fields.

We use the epochs where a visual stimulus was presented in the left or right visual field. These have the trigger codes `3` and `4`.

In [3]:
events = mne.find_events(raw)
epochs = mne.Epochs(raw, events, event_id=[3, 4])
evoked = epochs.average()

Let's plot the evoked fields:

In [ ]:
evoked.plot();


## Prepare beamforming

In the following, we will look at different aspects of beamforming. Let's first make sure we have our two main ingredients ready: the _forward model_ and the _data covariance matrix_.

Let's load the forward model - here, we load a _volumetric_ forward model. 

If you want to learn more about how to compute forward models, check out the notebook `Forward_modelling.ipynb`.

In [5]:
# load the precomputed MEG forward model from disk
fname_fwd = data_path / 'sample_audvis-meg-vol-7-fwd.fif'
fwd_meg = mne.read_forward_solution(fname_fwd)

For the computation of the beamformer, we also need a **data covariance matrix**. Let's compute our covariance matrix over the first 150 ms after the stimulus was presented.

In [6]:
# compute covariance matrix
data_cov = mne.compute_covariance(epochs, tmin=0.0, tmax=0.15, method='empirical')

## 1. The center-of-head bias

Let's first look at something that is called the center-of-head bias. This results from beamforming data _without_ using normalization of either the forward model or beamformer weight or using a contrast between conditions.

You will not come across this in MNE-Python with default parameter settings - by default, MNE-Python normalizes the beamformer weights, running a so-called _unit-noise-gain beamformer_. 
Let's first look at the center-of-head bias and then look at how **normalization** can mitigate it. 

<div class="alert alert-block alert-info">
    <b>Center-of-head bias</b>
    <br> <br>
      "The magnitude of forward field coefficients decreases with the distance of the dipole (its depth) to the sensors. [... T]he unit-gain beamformer weights, and hence the magnitude of the corresponding beamformer output, depends on the inverse of the forward field matrix at a given dipole location. This so-called center of head bias causes the reconstructed power of sources that are far away from the sensors to be a few orders of magnitude larger than the reconstructed power of more superficial sources. This places restrictions on a direct magnitude-based comparison of the beamformer output across space." <br>
      <br>
      Quote from: Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789
</div>


Let's compute the beamformer and apply it to the evoked fields. We choose the Linearly Constrained Minimum Variance beamformer (LCMV), which is expecting time-resolved data, such as an evoked field.

We use the following ingredients to compute the beamformer:
- _data covariance matrix_: This covariance matrix should represent the data.
- _the data_: The beamformer will be applied to this - we pass in our evoked object we have created above.
- _the forward model_: The one we loaded from disk.
- _orientation_: We ask the beamformer to compute the source orientations that maximize power for us. That is done using `pick_ori='max-power'`.

Crucially, we also choose the following setting:
- _weight_norm_: We choose `None` to not use any weight normalization. This corresponds to the _unit-gain_ beamformer.

In [20]:
from mne.beamformer import make_lcmv, apply_lcmv

filters = make_lcmv(epochs.info, fwd_meg, data_cov=data_cov, weight_norm=None, pick_ori='max-power')
stc = apply_lcmv(evoked=evoked, filters=filters)

We can plot the brain and time course:

In [ ]:
stc.crop(-0.05, 0.15).plot(subjects_dir=subjects_dir, subject='sample', src=fwd_meg['src']);

You can see that a lot of the activity is drawn towards the center/bottom of the brain - instead of showing up in the visual cortex as we would expect. The peak time does not match what we would expect from a visually evoked field either.

Let's now look at what we get if we compute a weight-normalized beamformer:

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> Change the weight norm parameter to 'unit-noise-gain-invariant'. What differences do you observe? </li>
    </ul>
</div>

## 2. Data covariance matrices and regularization

One ingredient the beamformer relies on to compute its weights is the data covariance matrix. It describes how channel information covaries across time. 

Certain procedures or circumstances can make the data (and thus the data covariance matrix) rank-deficient. These procedures and circumstances include:
- too few independent data points (short experiment, heavy data filtering)
- interpolation of channels
- cleaning with ICA or similar algorithms
- EEG (average) referencing

Let's have a closer look at what rank-deficiency means: the **rank** of a matrix is the maximal number of linearly independent columns of this matrix. If you remove (even noise) components from your data, you remove information, but not column or rows (there are still an equal amount of channels and time points present). This makes the data linearly dependent - or: rank deficient.

This can be visualized in MNE-Python easily when plotting the data covariance matrix. Let's visualize our covariance matrix!
In the singular value plots, you can see a vertical line at the estimated rank of the data. This vertical line coincides with the cliff of the singular value spectrum. 

<div class="alert alert-block alert-info">
    <b>Rank estimation of covariance matrices</b>
    <br> <br>
      "It is advisable to check the quality of the data covariance matrix by inspecting the rank and condition number, and also the singular value spectrum of the matrix, which is obtained by a Singular Value Decomposition (SVD). Plotting the singular values on a logarithmic axis provides an indication of the effective numerical rank of the covariance matrix. Typically, the singular value spectrum of an ill-conditioned matrix shows a "cliff" at its effective rank, with the numerically irrelevant components having singular values several orders of magnitude smaller than the rest. Sometimes, the spectrum can show two or more such cliffs, e.g., in the case of combined channel types or when the data has been processed with SSS. In such cases, a close inspection of the singular value spectrum is advisable, as conventional rank estimation via SVD can fail." <br>
      <br>
      Quote from: Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789
</div>


In [ ]:
mne.viz.plot_cov(data_cov, evoked.info);

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> By how much is the data rank-deficient? The number of MEG sensors is 102. </li>
      <li> Can you find out why the data is rank-deficient? Hint: inspect the info field of the evoked object. </li>
    </ul>
</div>

Why is this relevant? Well, during the beamformer computation, the inverse of the covariance matrix is taken - and that inverse is ill-defined if the matrix is rank-deficient. 

<div class="alert alert-block alert-info">
    <b>Rank estimation of covariance matrices</b>
    <br> <br>
      "If the estimate of the data covariance is unreliable, and thus proves to be ill-conditioned, a simple mathematical inverse of this matrix is either impossible, causing the beamformer computation to fail completely, or the used ill-conditioned covariance matrix will lead to poor beamformer results." <br>
      <br>
      Quote from: Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789
</div>


One way to deal with rank-deficient matrices is Tikhonov regularization. That de-correlates the columns of our rank-deficient covariance matrix by adding to the diagonal. The values to add to the diagonal are expressed as ratios/percentages of the sensor power. 

In MNE-Python, we use the `reg` parameter for that. By default, the algorithms uses a regularization parameter of 5%: `reg=0.05`. This is what we've silently been doing above. Let's set it to 0% and see what happens:

In [ ]:
filters = make_lcmv(epochs.info, fwd_meg, data_cov=data_cov, pick_ori='max-power', reg=0.0)
stc = apply_lcmv(evoked=evoked, filters=filters)
stc.crop(-0.05, 0.15).plot(subjects_dir=subjects_dir, subject='sample', src=fwd_meg['src']);

As we can see, not too much! This is because MNE-Python internally uses a pseudo-inverse, which will deal with small rank-deficiencies. 
If you want to see a beamformer fail dramatically due to rank-deficiency, head over to the `Working_with_SSSed_data.ipynb` notebook!
This notebook also details other ways to deal with rank-deficiency, e.g. spatial whitening and truncated pseudo inverses.

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> The downside of regularization is that it decreases spatial resolution. Can you support this claim by increasing the regularization by a lot? </li>
    </ul>
</div>

## 3. Scalar and vector beamformers

So far, we have been working with scalar beamformers. They give back one value per source location and time point. Vector beamformers give back three - for the _x_, _y_, and _z_ components of the dipole moment. 

The scalar beamformer in MNE-Python computes the source orientation per source location which maximizes power. That beamformer is computed by setting `pick_ori` to `'max-power'` as we did above.
Let now look at what a vector beamformer output looks like. For that, we first set `pick_ori` to `None`.

In [ ]:
filters = make_lcmv(epochs.info, fwd_meg, data_cov=data_cov, pick_ori=None)
stc = apply_lcmv(evoked=evoked, filters=filters)
stc.crop(-0.05, 0.15).plot(subjects_dir=subjects_dir, subject='sample', src=fwd_meg['src']);


This beamformer gives back only positive values. Indeed, looking at the help confirms that: `Orientations are pooled after computing a vector beamformer (Default).` Thus, the three orientations that were computed, were later combined into one value.

We, however, can also keep those three components separate by setting `pick_ori` to `vector`.

In [ ]:
filters = make_lcmv(epochs.info, fwd_meg, data_cov=data_cov, pick_ori='vector')
stc_vec = apply_lcmv(evoked=evoked, filters=filters)
stc_vec.crop(-0.05, 0.15).plot(subjects_dir=subjects_dir, subject='sample', src=fwd_meg['src']);

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> Compare this plot to the one above. Why is there no difference? </li>
    </ul>
</div>

We can look at the three components of the dipole. Let's do that for the peak voxel!

This requires some `matplotlib` code - a good exercise :)

In [ ]:
import matplotlib.pyplot as plt  # we will use Matplotlib for plotting

peak_vox, _ = stc_vec.get_peak(vert_as_index=True)  # get the peak voxel index

ori_labels = ["x ori", "y ori", "z ori"]  # labels for our plot

# loop across the orientations and plot
fig, ax = plt.subplots(1)
for ori, label in zip(stc_vec.data[peak_vox, :, :], ori_labels):
    # we indexed the peak voxel and stored the result in "ori":
    ax.plot(stc_vec.times, ori, label=f"{label} component")

# make plot neater:
ax.legend(loc="lower right")  # add legend
ax.set(
    xlabel="Time (s)",
    ylabel="Amplitude (a.u.)",
);  # labels

## 4. Common spatial filters

Often, we do not want to look at the time course of single condition, but compare conditions. Especially if we statistically compare them, it's important that we use _one_ spatial filter for the source reconstruction of both conditions. This beamformer should be computed using an equal amount of data from both conditions.

<div class="alert alert-block alert-info">
    <b>Common spatial filters</b>
    <br> <br>
      "If a contrast between two conditions is formed, it is good practice to compute the data covariance matrix based on an equal amount of data from both conditions (Gross et al., 2013). This common spatial filter is subsequently applied to each condition separately. This procedure prevents the spatial filter from being skewed and introducing spurious differences among the two conditions." <br>
      <br>
      Quote from: Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789
</div>


Let's compute a contrast between _activity_ (0.05 to 0.15 sec after stimulus onset) and _baseline_ (-0.15 t0 -0.05 sec relative to stimulus onset). We will compute two covariance matrices and then join them - and compute one _common_ spatial filter from that.
We will then apply this common spatial filter to the previously computed covariance matrices. Then we can contrast the obtained source power of _baseline_ and _activity_.

In [14]:
cov_base = mne.compute_covariance(epochs, tmin=-0.15, tmax=-0.05)
cov_acti = mne.compute_covariance(epochs, tmin=0.05, tmax=0.15)

# combine those two covariance matrices
cov_common = cov_base + cov_acti

We have learned that it's good practice to look at the covariance matrix:

In [ ]:
mne.viz.plot_cov(cov_common, evoked.info);

Let's now use this common data covariance matrix to create our beamformer:

In [17]:
filters = make_lcmv(epochs.info, fwd_meg, data_cov=cov_common, pick_ori='max-power')

Now the idea is to apply this spatial filter _twice_: once to the baseline data and once to the activity data. We will apply the spatial filter to the covariance matrices directly - this gives us source power, but no time series. 

It's of course also possible to apply the spatial filter to the evoked or epochs objects of the baseline and activity data periods!


In [18]:
from mne.beamformer import apply_lcmv_cov
stc_base = apply_lcmv_cov(cov_base, filters)
stc_acti = apply_lcmv_cov(cov_acti, filters)

Now we can compute a contrast between the two source estimates we have obtained. Let's use the baseline estimate to normalize the activity. And then let's plot the result.

In [ ]:
stc_diff = stc_acti.copy()  # we make a copy so you can use the original for further exploration
stc_diff /= stc_base

stc_diff.plot(subjects_dir=subjects_dir, subject='sample', src=fwd_meg['src']);

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> Can you plot the difference between active and baseline instead? </li>
    </ul>
</div>

## Further reading

A whole section of Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789 is dedicated to "Practical considerations and best practices in beamformer source reconstruction".

